In [ ]:
# Setup fake data generator
from faker import Faker
fake = Faker()

In [ ]:
# setup db connection
import mysql.connector

cnx = cnx = mysql.connector.connect(user='reslifeadmin',password='eoekK8bRe4wa',
                                    host='csdb.wheaton.edu',
                                    database='reslife')
cursor = cnx.cursor()

In [ ]:
# get current db data
cursor.execute('SELECT * FROM t_students ORDER BY studentId')
students = cursor.fetchall()

In [ ]:
# function to get a new student id that doesn't already exist in the db table
def get_stuid():
    while True:
        stuid = fake.random_number(digits=5,fix_len=True)
        cursor.execute('SELECT * FROM t_students WHERE studentId=%(stuid)s', {'stuid': stuid})
        if not cursor.fetchall():
            return stuid

In [ ]:
# iterate over current db rows, updating them with fake data
for stu in students:
    # setup fake data to be used in db update

    # names and email
    fn = fake.first_name()
    preferred_fn = ''
    if stu[3] and stu[3] != '':
        preferred_fn = fake.first_name()
    ln = fake.last_name()
    email = (fn.lower() if preferred_fn == '' else preferred_fn.lower()) + '.' + ln.lower() + '@my.wheaton.edu'

    # birthdate
    # considering that grad students live in college housing, we'll assume
    # that college students are >= 18 and <= 100 years old
    # never stop learning, right?! :)
    dob = fake.date_of_birth(minimum_age=18, maximum_age=70)

    # address
    city = fake.city()
    st = fake.state_abbr()

    # id number
    # new id must not already be in the table
    oldid = stu[10]
    stuid = get_stuid()
    
    # sql update
    update = 'UPDATE t_students SET name_last=%(ln)s, name_first=%(fn)s, date_of_birth=%(dob)s, email=%(email)s, ' \
            'city=%(city)s, state_province=%(st)s, studentID=%(stuid)s, name_preferred=%(preferred_fn)s ' \
            'WHERE studentID=%(oldid)s'
    params = { 'ln': ln, 'fn': fn, 'dob': dob, 'city': city, 'st': st, 'stuid': stuid, 'oldid': oldid, 
              'preferred_fn': preferred_fn, 'email': email}

    cursor.execute(update, params)
    print cursor.statement
    
    cnx.commit()

In [ ]:
# cleanup
cursor.close()
cnx.close()